In [8]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import faulthandler
faulthandler.enable()

### Three different influencer types
1. Health & Wellness: I am focused on improving the health and well-being of my community. I do this by sharing diet, exercise, and lifestyle tips.
2. Food & Drink: Discovering hidden gems in the food insustry are my passion. I love to uncover good and unique food to share with my followers.
3. Travel Influencer: My passion is exploring the globe. I love experiencing different cultures from their foods to day-to-day lifestyle. I try to share these unique experiences with my following

In [15]:
# Define the three different influencers with a potentialto be included in the campaign
influencer_list = {'Martha': 'Discovering hidden gems in the food insustry are my passion. I love to uncover good and unique food to share with my followers.',
                   'Billy': 'My passion is exploring the globe. I love experiencing different cultures from their foods to day-to-day lifestyle. I try to share these unique experiences with my following',
                   'Joan': 'I am focused on improving the health and well-being of my community. I do this by sharing diet, exercise, and lifestyle tips.'}

# Add these infuencers to a pandas object
influencers = pd.DataFrame({'name': list(influencer_list.keys()), 'description': list(influencer_list.values())})

## Encode the influencers in to an "in-memory" vector storage DB

In [16]:
# Extract the desctiptions from the influencers
descriptions = influencers['description']
# Define the encoder which will be used to create "embeddings"
encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
# Embed the descriptions into vectors
vectors = encoder.encode(descriptions)

# Identify thq dimension of the vectors
vector_dimension = vectors.shape[1]
# Create an index for the influencervectors
index = faiss.IndexFlatL2(vector_dimension)
# Normalize the vectors
faiss.normalize_L2(vectors)
# Add the vectors to the index
index.add(vectors)

## Encode a campaign description which will be used to search the influencers

In [35]:
# Define a query which will be used to search for similar influencers
marketing_campaign_description = 'A salad campaign'
# Encode the campaign description which will be used as the search vector for 
search_vector = encoder.encode(marketing_campaign_description)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

## Execute the index search

In [36]:
# Define the number of similar influencers to return
k = index.ntotal
# Execute the search vector
distances, ann = index.search(_vector, k=k)

## Display the results

In [37]:
# Create a pandas object with the results
results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
# Merge the results with the influencers
merge = pd.merge(results, influencers, left_on='ann', right_index=True)
merge